In [1]:
import sys
if "../" not in sys.path: sys.path.insert(0,"../");

## Building utilities for marynlp

Creating multiple utilities for building models, downloading the files and a tone of other things

In [2]:
## Module downloading sequence

In [3]:
## Using the SED Morpheme template
from marynlp.utils.storage import download as dl
from marynlp.utils import storage

# setup the download bucket
bucket = storage.get_bucket("../resources/mary_africa_credentials_key.json", "marynlp-private")

morpheme_template_file = dl.file_from_google_temporary("models/sed_morpheme_template.txt", bucket); morpheme_template_file

'/tmp/tmpntdygnkv/models/sed_morpheme_template.txt'

In [8]:
from experimental.sed import MorphologyAnalyzer
from typing import List, Tuple

class WordBreaker(object):
    def __init__(self, ma: MorphologyAnalyzer):
        self.ma_ = ma
        
    def break_word(self, word: str) -> Tuple[str]:
        return tuple([ su for su in self.ma_.break_text([word])[word]])
    
analyzer = MorphologyAnalyzer(morpheme_template_file); analyzer
bk = WordBreaker(analyzer)
bk.break_word("walisema")

('wa', 'lis', 'ma')